In [ ]:
import gcsfs
import google.auth
import modin.config as cfg
import modin.pandas as pd
from distributed import Client
from google.colab import auth
from modin.config import ProgressBar

cfg.Engine.put("dask")
ProgressBar.enable()

import wandb
from tqdm.notebook import tqdm

import google.auth
from google.colab import auth
import gcsfs


In [2]:
import pandas as pd

In [32]:
buy_sell= pd.read_csv("../data/raw/matched_cboe_quotes.csv", usecols=["buy_sell"])

print(len(buy_sell))

27092257


In [4]:
df = pd.read_csv("../data/raw/matched_cboe_quotes.csv", nrows=20)
df

,UNDERLYING_SYMBOL,QUOTE_DATETIME,SEQUENCE_NUMBER,ROOT,EXPIRATION,STRK_PRC,OPTION_TYPE,EXCHANGE,TRADE_SIZE,TRADE_PRICE,...,bid_size_ex,ask_size_ex,date,price_all_lead,price_all_lag,optionid,day_vol,price_ex_lead,price_ex_lag,buy_sell
0,A,03JAN11:14:49:37,148135651,A,22JAN2011,35,C,5,16,7.13,...,52.0,38.0,03JAN2011,7.00,6.53,46548099,30,7.89,6.53,1
1,A,03JAN11:14:49:37,148135675,A,22JAN2011,35,C,5,14,7.13,...,52.0,38.0,03JAN2011,7.00,6.53,46548099,30,7.89,6.53,1
2,A,03JAN11:14:17:17,136537180,A,22JAN2011,40,C,5,5,2.30,...,5.0,43.0,03JAN2011,1.64,2.00,45931458,5,1.84,2.01,1
3,A,03JAN11:12:33:45,108068163,A,19FEB2011,39,C,5,3,3.70,...,199.0,11.0,03JAN2011,3.35,3.68,49449150,3,3.20,3.68,-1
4,A,03JAN11:10:42:58,62936607,A,19FEB2011,39,P,5,7,0.73,...,10.0,34.0,03JAN2011,0.75,0.75,36266478,15,0.75,0.75,-1
5,A,03JAN11:10:15:19,42958899,A,19FEB2011,39,P,5,5,0.75,...,11.0,95.0,03JAN2011,0.73,0.90,36266478,15,0.73,1.38,-1
6,A,03JAN11:10:42:58,62936917,A,19FEB2011,39,P,5,3,0.73,...,10.0,34.0,03JAN2011,0.75,0.75,36266478,15,0.75,0.75,-1
7,A,03JAN11:15:49:33,170610855,A,19FEB2011,40,C,5,4,3.01,...,37.0,247.0,03JAN2011,2.83,3.10,35937048,22,2.74,3.08,1
8,A,03JAN11:10:23:01,49694297,A,19FEB2011,40,C,5,8,3.08,...,267.0,96.0,03JAN2011,2.97,2.67,35937048,22,3.01,2.67,1
9,A,03JAN11:09:30:34,1150506,A,19FEB2011,40,C,5,10,2.67,...,NaN,NaN,03JAN2011,3.08,2.85,35937048,22,3.08,2.73,1


In [15]:
# User inputs
fname = "../data/raw/matched_cboe_quotes.csv"
tail_len = 20

# The two steps in the description
n_rows = sum(1 for _ in open(fname, 'r'))
print(n_rows)
df_tail = pd.read_csv(fname, skiprows=n_rows - tail_len)

# df = pd.read_csv("../data/raw/matched_cboe_quotes.csv", nrows=20)
# df

27092258


In [17]:
df_tail.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX
28SEP11:12:39:12,28SEP11:12:09:46,28SEP11:11:55:59,28SEP11:12:38:50,28SEP11:12:38:06,28SEP11:12:37:51,28SEP11:12:33:49,28SEP11:12:33:23,28SEP11:12:10:54,28SEP11:13:28:54,28SEP11:13:27:50,28SEP11:13:29:08,28SEP11:13:30:22,28SEP11:13:27:43,28SEP11:13:04:27,28SEP11:13:21:18,28SEP11:13:27:43,28SEP11:11:54:38,28SEP11:13:02:51,28SEP11:13:21:18
128405364,114663190,107509084,128373522,128068111,127957981,126487688,126339557,115312357,149215018,148635491,149462403,150065175,148517868,138821776,146175658,148518625,106691856,138224544,146175636
SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX
22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011
1410,1415,1415,1415,1415,1415,1415,1415,1415,1450,1450,1460,1460,1500,1500,1520,1520,1575,1575,1575
C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C
5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
2,9,3,1,7,7,9,9,9,9,9,9,9,9,9,9,9,3,9,9
0.06,0.05,0.05,0.06,0.06,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05


In [28]:
n_rows = 27092258
#n_rows = 37155412
df_tail = pd.read_csv(fname, skiprows=(n_rows - 25), dtype=object)

In [29]:
df_tail.T

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,...,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX
28SEP11:16:10:43,28SEP11:16:10:43,28SEP11:16:10:43,28SEP11:15:51:27,28SEP11:15:51:27,28SEP11:12:39:12,28SEP11:12:09:46,28SEP11:11:55:59,28SEP11:12:38:50,28SEP11:12:38:06,28SEP11:12:37:51,...,28SEP11:13:27:50,28SEP11:13:29:08,28SEP11:13:30:22,28SEP11:13:27:43,28SEP11:13:04:27,28SEP11:13:21:18,28SEP11:13:27:43,28SEP11:11:54:38,28SEP11:13:02:51,28SEP11:13:21:18
243810226,243810208,243810212,233921714,233921724,128405364,114663190,107509084,128373522,128068111,127957981,...,148635491,149462403,150065175,148517868,138821776,146175658,148518625,106691856,138224544,146175636
SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,...,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX
22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,...,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011
1360,1360,1360,1365,1370,1410,1415,1415,1415,1415,1415,...,1450,1460,1460,1500,1500,1520,1520,1575,1575,1575
C,C,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
5,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
23,23,27,15,15,2,9,3,1,7,7,...,9,9,9,9,9,9,9,3,9,9
0.1,0.1,0.1,0.14,0.09,0.06,0.05,0.05,0.06,0.06,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05


In [30]:
df_tail.set_axis(df.columns, axis=1, copy=False)

,UNDERLYING_SYMBOL,QUOTE_DATETIME,SEQUENCE_NUMBER,ROOT,EXPIRATION,STRK_PRC,OPTION_TYPE,EXCHANGE,TRADE_SIZE,TRADE_PRICE,...,bid_size_ex,ask_size_ex,date,price_all_lead,price_all_lag,optionid,day_vol,price_ex_lead,price_ex_lag,buy_sell
0,^SPX,28SEP11:16:10:43,243810208,SPX,22OCT2011,1360,C,5,23,0.1,...,314,100,28SEP2011,0.05,0.15,63099317,228,0.05,0.15,1
1,^SPX,28SEP11:16:10:43,243810212,SPX,22OCT2011,1360,C,5,27,0.1,...,314,100,28SEP2011,0.05,0.15,63099317,228,0.05,0.15,1
2,^SPX,28SEP11:15:51:27,233921714,SPX,22OCT2011,1365,C,5,15,0.14,...,10,50,28SEP2011,0.1,0.25,63099318,15,0.1,0.25,1
3,^SPX,28SEP11:15:51:27,233921724,SPX,22OCT2011,1370,C,5,15,0.09,...,10,50,28SEP2011,0.05,0.15,63099319,15,0.05,0.15,-1
4,^SPX,28SEP11:12:39:12,128405364,SPX,22OCT2011,1410,C,5,2,0.06,...,0,22,28SEP2011,0.05,0.1,63099327,2,0.05,0.1,-1
5,^SPX,28SEP11:12:09:46,114663190,SPX,22OCT2011,1415,C,5,9,0.05,...,0,50,28SEP2011,0.06,0.2,63099328,54,0.06,0.2,-1
6,^SPX,28SEP11:11:55:59,107509084,SPX,22OCT2011,1415,C,5,3,0.05,...,0,50,28SEP2011,0.06,0.2,63099328,54,0.06,0.2,-1
7,^SPX,28SEP11:12:38:50,128373522,SPX,22OCT2011,1415,C,5,1,0.06,...,0,50,28SEP2011,NaN,0.05,63099328,54,NaN,0.05,-1
8,^SPX,28SEP11:12:38:06,128068111,SPX,22OCT2011,1415,C,5,7,0.06,...,0,50,28SEP2011,NaN,0.05,63099328,54,NaN,0.05,-1
9,^SPX,28SEP11:12:37:51,127957981,SPX,22OCT2011,1415,C,5,7,0.05,...,0,50,28SEP2011,0.06,0.2,63099328,54,0.06,0.2,-1


In [31]:
df_tail.T

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,...,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX,^SPX
28SEP11:16:10:43,28SEP11:16:10:43,28SEP11:16:10:43,28SEP11:15:51:27,28SEP11:15:51:27,28SEP11:12:39:12,28SEP11:12:09:46,28SEP11:11:55:59,28SEP11:12:38:50,28SEP11:12:38:06,28SEP11:12:37:51,...,28SEP11:13:27:50,28SEP11:13:29:08,28SEP11:13:30:22,28SEP11:13:27:43,28SEP11:13:04:27,28SEP11:13:21:18,28SEP11:13:27:43,28SEP11:11:54:38,28SEP11:13:02:51,28SEP11:13:21:18
243810226,243810208,243810212,233921714,233921724,128405364,114663190,107509084,128373522,128068111,127957981,...,148635491,149462403,150065175,148517868,138821776,146175658,148518625,106691856,138224544,146175636
SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,...,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX,SPX
22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,...,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011,22OCT2011
1360,1360,1360,1365,1370,1410,1415,1415,1415,1415,1415,...,1450,1460,1460,1500,1500,1520,1520,1575,1575,1575
C,C,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
5,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
23,23,27,15,15,2,9,3,1,7,7,...,9,9,9,9,9,9,9,3,9,9
0.1,0.1,0.1,0.14,0.09,0.06,0.05,0.05,0.06,0.06,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05


In [38]:
df = pd.read_csv("../data/raw/livevol_ise_raw.csv", nrows=20)
df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
UNDERLYING_SYMBOL,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A
QUOTE_DATETIME,24OCT13:12:30:37,24OCT13:12:30:37,24OCT13:14:26:29,24OCT13:14:26:29,24OCT13:09:53:47,24OCT13:15:01:24,24OCT13:14:26:30,24OCT13:14:26:29,24OCT13:15:59:48,24OCT13:14:26:29,24OCT13:14:26:29,24OCT13:14:26:29,24OCT13:14:26:29,24OCT13:11:09:29,24OCT13:15:59:48,24OCT13:15:33:59,24OCT13:14:26:30,24OCT13:14:26:29,24OCT13:14:26:29,24OCT13:14:26:29
SEQUENCE_NUMBER,2074528803,2074528801,2774915009,2774905503,436146800,2996770500,2775005101,2774922015,3428613701,2774905501,2774924709,2774920007,2774909800,1251662800,3428613702,3191563105,2775005100,2774921711,2774923720,2774921606
ROOT,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A
EXPIRATION,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013,16NOV2013
STRK_PRC,44.0,46.0,47.0,48.0,49.0,49.0,49.0,49.0,49.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,52.5,52.5
OPTION_TYPE,P,P,P,P,C,P,P,P,P,C,C,C,C,C,P,P,P,P,C,C
EXCHANGE,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
TRADE_SIZE,1,1,10,40,1,5,1,10,1,40,25,14,59,2,1,2,1,10,1,15
TRADE_PRICE,0.13,0.27,0.36,0.52,2.63,0.73,0.72,0.75,0.71,2.17,2.16,2.16,2.18,2.21,1.01,1.02,1.05,1.07,0.9,0.93


In [36]:
# User inputs
fname = "../data/raw/livevol_ise_raw.csv"
tail_len = 20

# The two steps in the description
n_rows = sum(1 for _ in open(fname, 'r'))
print(n_rows)
df_tail = pd.read_csv(fname, skiprows=n_rows - tail_len)

90976083


MemoryError: 

In [34]:
necessary_cols = [
        "QUOTE_DATETIME",
        "ROOT",
        "EXPIRATION",
        "STRK_PRC",
        "OPTION_TYPE",
        "TRADE_SIZE",
        "TRADE_PRICE",
        "BEST_BID",
        "BEST_ASK",
        "order_id",
        "ask_ex",
        "bid_ex",
        "bid_size_ex",
        "ask_size_ex",
        "date",
        "price_all_lead",
        "price_all_lag",
        "optionid",
        "day_vol",
        "price_ex_lead",
        "price_ex_lag",
        "buy_sell",
        "issue_type",
        "myn",
        "secid_OM",
]

In [35]:
for col in necessary_cols:
    if col not in df.columns:
        print(col)

day_vol
price_ex_lead
price_ex_lag
buy_sell
issue_type
myn
secid_OM


In [ ]:
# connect to weights and biases
run = wandb.init(project="thesis", job_type="dataset-creation", entity="fbv")
dataset = wandb.Artifact(name="raw_data", type="raw_data")


In [ ]:
# connect to google cloud storage. Auth method requires to be on colab.
auth.authenticate_user()
credentials, _ = google.auth.default()
fs = gcsfs.GCSFileSystem(project="thesis", token=credentials)


In [ ]:
file_path_input = "gs://thesis-bucket-option-trade-classification/data/raw/matched_samples_ise_quotes_extended.csv"
file_path_output = "gs://thesis-bucket-option-trade-classification/data/preprocessed/"


In [ ]:
def import_data(input_file: str) -> pd.DataFrame:
    """
    create a dataframe and optimize its memory usage.

    I. e., apply some optimizations i.e, manual inference of dtypes, pre-selection
    of unique columns and chunking to enable import.

    Adapted from here:
    https://pandas.pydata.org/pandas-docs/stable/user_guide/scale.html

    Args:
        input_file (str): input file.

    Returns:
        pd.DataFrame: dataframe.
    """
    dfs = []
    chunksize = 10**6

    # do not load 'CANCELED_TRADE_CONDITION_ID' and 'EXCHANGE'
    usecols = [
        "QUOTE_DATETIME",
        "ROOT",
        "EXPIRATION",
        "STRK_PRC",
        "OPTION_TYPE",
        "TRADE_SIZE",
        "TRADE_PRICE",
        "BEST_BID",
        "BEST_ASK",
        "order_id",
        "ask_ex",
        "bid_ex",
        "bid_size_ex",
        "ask_size_ex",
        "date",
        "price_all_lead",
        "price_all_lag",
        "optionid",
        "day_vol",
        "price_ex_lead",
        "price_ex_lag",
        "buy_sell",
        "ASK_1",
        "ASK_2",
        "ASK_3",
        "ASK_4",
        "ASK_5",
        "ASK_6",
        "ASK_7",
        "ASK_8",
        "ASK_9",
        "ASK_10",
        "ASK_11",
        "ASK_12",
        "ASK_13",
        "ASK_14",
        "ASK_15",
        "ASK_16",
        "BID_1",
        "BID_2",
        "BID_3",
        "BID_4",
        "BID_5",
        "BID_6",
        "BID_7",
        "BID_8",
        "BID_9",
        "BID_10",
        "BID_11",
        "BID_12",
        "BID_13",
        "BID_14",
        "BID_15",
        "BID_16",
        "issue_type",
        "myn",
        "secid_OM",
    ]

    dtypes = {
        "QUOTE_DATETIME": "object",
        "ROOT": "category",
        "EXPIRATION": "object",
        "STRK_PRC": "float32",
        "OPTION_TYPE": "category",
        "EXCHANGE": "int64",
        "TRADE_SIZE": "int64",
        "TRADE_PRICE": "float32",
        "CANCELED_TRADE_CONDITION_ID": "int64",
        "BEST_BID": "float32",
        "BEST_ASK": "float32",
        "order_id": "int64",
        "ask_ex": "float32",
        "bid_ex": "float32",
        "bid_size_ex": "float32",
        "ask_size_ex": "float32",
        "date": "object",
        "price_all_lead": "float32",
        "price_all_lag": "float32",
        "optionid": "float32",
        "day_vol": "float32",
        "price_ex_lead": "float32",
        "price_ex_lag": "float32",
        "buy_sell": "int8",
        "ASK_1": "float32",
        "ASK_2": "float32",
        "ASK_3": "float32",
        "ASK_4": "float32",
        "ASK_5": "float32",
        "ASK_6": "float32",
        "ASK_7": "float32",
        "ASK_8": "float32",
        "ASK_9": "float32",
        "ASK_10": "float32",
        "ASK_11": "float32",
        "ASK_12": "float32",
        "ASK_13": "float32",
        "ASK_14": "float32",
        "ASK_15": "float32",
        "ASK_16": "float32",
        "BID_1": "float32",
        "BID_2": "float32",
        "BID_3": "float32",
        "BID_4": "float32",
        "BID_5": "float32",
        "BID_6": "float32",
        "BID_7": "float32",
        "BID_8": "float32",
        "BID_9": "float32",
        "BID_10": "float32",
        "BID_11": "float32",
        "BID_12": "float32",
        "BID_13": "float32",
        "BID_14": "float32",
        "BID_15": "float32",
        "BID_16": "float32",
        "issue_type": "category",
        "myn": "float32",
        "secid_OM": "float32",
    }

    # log raw file in w & b
    dataset.add_reference(input_file, name="raw_csv")

    with pd.read_csv(
        input_file, chunksize=chunksize, usecols=usecols, dtype=dtypes
    ) as reader:
        for chunk in reader:
            dfs.append(chunk)

    df = pd.concat(dfs, axis=0)

    format = "%d%b%Y"
    df["EXPIRATION"] = pd.to_datetime(df["EXPIRATION"], format=format)
    df["date"] = pd.to_datetime(df["date"], format=format)

    format = "%d%b%y:%H:%M:%S"
    df["QUOTE_DATETIME"] = pd.to_datetime(df["QUOTE_DATETIME"], format=format)
    return df


In [ ]:
def df_to_parquet(
    df: pd.DataFrame, target_dir: str, chunk_size: int = 1000000, **parquet_wargs
) -> None:
    """
    Write pd.DataFrame to parquet format.

    Args:
        df (pd.DataFrame): input dataframe.
        target_dir (str): local directory where parquet files are written to.
        chunk_size (int, optional): number of rows stored in one chunk of parquet file.
        Defaults to 1000000.
    """
    for i in tqdm(range(0, len(df), chunk_size)):
        slc = df.iloc[i : i + chunk_size]
        chunk = int(i / chunk_size)
        output_path = (
            target_dir
            + f"matched_ise_quotes_min_mem_usage_extended_part_{chunk:04d}.parquet"
        )
        slc.to_parquet(output_path, **parquet_wargs)

        # log in w & b
        dataset.add_reference(output_path, name=f"raw_parquet_{chunk:04d}")


In [ ]:
client = Client()
df = import_data(file_path_input)
df_to_parquet(df, file_path_output)


In [ ]:
# Log the artifact to save it as an output of this run
run.log_artifact(dataset)
wandb.finish()
